In [1]:
import config, data_load, data_info, data_plot
import plotly.express as px
import seaborn as sns
import sqlite3
import pandas as pd
import numpy as np

In [3]:
path_filters = 'queries/info/filters.sql'
with open(path_filters) as f:
	fitlers_query = f.read()
df = data_load.df_query(fitlers_query)
# res = data_load.sql_query(fitlers_query)
# res.json()['rows'][0][0]
df

,RACA_COR,count(RACA_COR)
0,03,7107874
1,99,4808111
2,01,4054120
3,02,559804
4,04,220957
5,05,65320


In [32]:
from plotly import graph_objects as go

fig = go.Figure(go.Funnel(
	# y = ['PROC_REA Partos', 'ano_internacao entre 2010-19', 'res_RSAUDCOD válido', 'res_SIGLA_UF sem DF', 'IDADE > 10', 'CNES válido', 'raça/cor preenchida'],
	# x = [23302307, 17109256, 17106725, 16817215, 16816186, 16816186, 12008075],
	y = ['PROC_REA Partos entre 2010-19', 'res_RSAUDCOD válido', 'res_SIGLA_UF sem DF', 'IDADE > 10'],
	x = [17109256, 17106725, 16817215, 16816186],
	textposition = "inside",
	textinfo = "value+percent initial",
	# opacity = 0.65, marker = {"color": ["deepskyblue", "lightsalmon", "tan", "teal", "silver"],
	# "line": {"width": [4, 2, 2, 3, 1, 1], "color": ["wheat", "wheat", "blue", "wheat", "wheat"]}},
	# connector = {"line": {"color": "royalblue", "dash": "dot", "width": 3}})
	))

fig.show()

In [4]:
dasdas

NameError: name 'dasdas' is not defined

In [ ]:
cnx = sqlite3.connect(config.PATH_DB)

In [ ]:
path_year_percent = 'queries/info/grouped.sql'
with open(path_year_percent) as f:
	query = f.read()
df = pd.read_sql_query(query, cnx)
df

,ano,parto,regiao,uf,grupo,mov_municipio,mov_regiao_saude,records
0,2010,NORMAL,Sudeste,SP,2,0,0,53387
1,2010,NORMAL,Sudeste,SP,1,0,0,37651
2,2010,CESARIANO,Sudeste,SP,1,0,0,26355
3,2010,CESARIANO,Sudeste,SP,2,0,0,24590
4,2010,NORMAL,Sudeste,SP,3,0,0,23676
...,...,...,...,...,...,...,...,...
7386,2019,NORMAL,Norte,AC,4,1,1,1
7387,2019,NORMAL,Norte,AP,2,1,1,1
7388,2019,NORMAL,Norte,AP,5,1,1,1
7389,2019,NORMAL,Norte,RR,1,1,1,1


In [ ]:
def group_all_but(df, but='records'):
	return df.groupby(
		by=[col for col in df.columns if col != but],
		as_index=False
	).sum().sort_values(
		by=['ano', 'records'],
		ascending=[True, False],
	)

In [ ]:
def get_mov_between(df, mov_between):
	cols = [col
		for col in df.columns if col[:3] != 'mov']
	dff = df[cols].copy()
	dff['mov_between'] = mov_between
	dff = group_all_but(dff)
	return dff

In [ ]:
def group_mov_between(df):
	return pd.concat([
		get_mov_between(df, 'all'),
		get_mov_between(df[df['mov_municipio'] == 1], 'municipal'),
		get_mov_between(df[df['mov_regiao_saude'] == 1], 'regional')
	]).reset_index(drop=True)

In [ ]:
def separate_movs_in_dict(df):
	d = dict()
	movs = df['mov_between'].unique()
	cols = [col for col in df.columns if col[:3] != 'mov']
	for mov in movs:
		dff = df[
			df['mov_between'] == mov
		][cols].set_index(['ano', 'parto'])
		d[mov] = dff
	return d

In [ ]:
def join_dfs(d, left, right):
	df = d[left].join(
		d[right],
		lsuffix='_num',
		rsuffix='_den',
	)
	df['mov_between'] = left
	df['percent'] = 100. * df['records_num'] / df['records_den']
	df = df.reset_index()
	return df

In [ ]:
def get_frac(df):
	d = separate_movs_in_dict(df)
	return pd.concat([
		join_dfs(d, 'municipal', 'all'),
		join_dfs(d, 'regional', 'municipal')
	])

In [ ]:
def group_col(df, column=None, value=None):
	del_cols = ['regiao', 'uf', 'grupo']
	if column:
		del_cols = [col
			for col in del_cols
				if col != column]
	cols_filter = [col for col in df.columns
			if col not in del_cols]
	dff = df[cols_filter].copy()
	if value:
		dff = dff[dff[column] == value]
	dff = group_all_but(dff)
	return get_frac(dff)

In [ ]:
df_s = group_mov_between(df)
df_s

,ano,parto,regiao,uf,grupo,mov_between,records
0,2010,NORMAL,Sudeste,SP,2,all,57558
1,2010,NORMAL,Sudeste,SP,1,all,45426
2,2010,CESARIANO,Sudeste,SP,1,all,31318
3,2010,NORMAL,Sudeste,SP,3,all,28810
4,2010,CESARIANO,Sudeste,SP,2,all,27666
...,...,...,...,...,...,...,...
7515,2019,NORMAL,Norte,AC,4,regional,1
7516,2019,NORMAL,Norte,AP,2,regional,1
7517,2019,NORMAL,Norte,AP,5,regional,1
7518,2019,NORMAL,Norte,RR,1,regional,1


In [ ]:
def plot(df):
	return px.scatter(df,
		x='ano',
		y='percent',
		color='parto',
		symbol='mov_between',
		trendline="ols",
	)

In [ ]:
df = group_col(df_s,
	column = 'grupo',
	value  = 5,
)
fig = plot(df)
fig.show()

In [ ]:
results = px.get_trendline_results(fig)
results['px_fit_results'][0].params

array([-3.3518335e+02,  1.7897567e-01])

In [ ]:
dt = pd.DataFrame([r.params for r in results['px_fit_results']])
dt

,0,1
0,-335.183350,0.178976
1,-1335.548284,0.670490
2,294.708144,-0.132769
3,-453.942965,0.235088


In [ ]:
px.scatter(dt,x=1,y=0)

In [ ]:
for r in results:
	print(r)

parto
mov_between
px_fit_results


In [ ]:
df
fig = plot(df)
fig.show()

In [ ]:
df = group_col(df_s,
	column = 'regiao',
	value  = 'Norte',
)

In [ ]:
df
fig = plot(df)
fig.show()

In [ ]:
df = group_col(df_s,
	column = 'regiao',
	value  = None,
)
df

,ano,parto,regiao_num,records_num,regiao_den,records_den,mov_between,percent
0,2010,CESARIANO,Nordeste,47286,Sudeste,153960,municipal,30.713172
1,2010,CESARIANO,Nordeste,47286,Nordeste,116913,municipal,40.445459
2,2010,CESARIANO,Nordeste,47286,Sul,63946,municipal,73.946768
3,2010,CESARIANO,Nordeste,47286,Norte,43460,municipal,108.803497
4,2010,CESARIANO,Nordeste,47286,Centro-Oeste,38870,municipal,121.651659
...,...,...,...,...,...,...,...,...
495,2019,NORMAL,Sul,3487,Nordeste,80531,regional,4.330010
496,2019,NORMAL,Sul,3487,Sudeste,59801,regional,5.831006
497,2019,NORMAL,Sul,3487,Centro-Oeste,47001,regional,7.418991
498,2019,NORMAL,Sul,3487,Sul,29417,regional,11.853690
